In [24]:
import requests
import datetime
import json
import os

In [3]:
url="https://player.twitch.tv/?autoplay=true&muted=false&parent=hsreplay.net&t=3h31m7s&video=v2156055583"
response=requests.get(url)

In [21]:
#gather data from the official HS website.
auth_key="KRlgrmP7M8drTyLaYsNYz39GEqco5G1RUq"
headers={"Authorization": "Bearer {:s}".format(auth_key)}
date_string=str(datetime.datetime.now().date())

In [23]:
#for page_index in range(15, 22):
#   page_url="https://us.api.blizzard.com/hearthstone/cards?locale=en_US&set=standard&pageSize=50&page={:d}".format(page_index)
#    response=requests.get(page_url, headers=headers)
#    response_data=response.json()
#    #save the json
#    with open("./hs_card_data/{:s}-page_{:d}.json".format(date_string, page_index), 'w') as file:
#        json.dump(response_data, file)

In [59]:
#read hearthstone card data
all_cards=[]
card_data_path="./hs_card_data/2024-05-28/"
for filename in os.listdir(card_data_path):
    filepath=os.path.join(card_data_path, filename)
    with open(filepath, 'r') as file:
        hs_page_data=json.load(file)
    page=hs_page_data['page']
    cards=hs_page_data['cards']
    all_cards.extend(cards)

id_to_card={}
for card in all_cards:
    id_to_card[card['id']]=card

In [61]:
id_list=list(set(id_to_card))
print(sorted(id_list))

[69632, 94210, 69635, 69634, 102405, 69638, 102407, 69640, 69641, 94218, 69643, 102408, 69645, 69637, 94225, 102418, 69649, 69652, 102421, 102422, 69655, 98328, 102424, 98330, 102427, 102426, 69661, 69658, 69663, 102432, 69660, 69666, 102434, 69668, 69665, 69662, 102435, 94248, 94249, 69674, 69675, 102443, 69677, 94251, 98344, 94250, 102449, 69679, 69683, 81972, 69684, 69688, 69691, 102463, 69698, 69700, 69702, 69703, 102472, 98377, 69706, 69705, 69707, 98382, 69713, 98389, 98390, 69718, 98394, 69726, 69731, 98403, 69735, 98410, 98411, 69740, 98413, 69742, 98417, 98427, 98434, 82072, 106652, 98468, 98469, 94381, 86190, 106673, 106697, 98520, 86235, 90356, 90359, 90366, 98560, 90368, 98565, 102719, 98626, 102724, 90439, 90440, 102728, 90441, 102732, 102733, 102741, 102745, 98652, 90461, 90463, 90481, 90484, 98680, 102784, 90497, 102790, 98701, 102799, 98704, 90532, 90534, 90541, 98734, 90543, 98750, 90561, 98762, 90572, 98768, 98769, 90583, 90586, 90591, 90592, 90600, 90603, 90606, 9060

In [53]:
#open up some log files
log_path="/Applications/Hearthstone/Logs/Hearthstone_2024_05_28_11_10_23"

In [54]:
#power.log file and zone.log file combined contains all the meaningful data for the game.
#HDT - interpretable data.
#open power.log
with open(os.path.join(log_path, "Power.log"), 'r') as file:
    power_log=file.readlines()

#open zone.log
with open(os.path.join(log_path, "Zone.log"), 'r') as file:
    zone_log=file.readlines()

In [57]:
for power_logline in power_log[1000:1100]:
    print(power_logline)

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -     TAG_CHANGE Entity=Claasix#3609 tag=2166 value=39 

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -     TAG_CHANGE Entity=Claasix#3609 tag=467 value=4 

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -     SHOW_ENTITY - Updating Entity=[entityName=UNKNOWN ENTITY [cardType=INVALID] id=39 zone=DECK zonePos=0 cardId= player=2] CardID=TTN_469

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -         tag=CONTROLLER value=2

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -         tag=CARDTYPE value=MINION

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -         tag=TAG_LAST_KNOWN_COST_IN_HAND value=2

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -         tag=479 value=2

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -         tag=COST value=2

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -         tag=ATK value=2

D 11:11:59.7602130 PowerTaskList.DebugPrintPower() -         tag=HEALTH value=2

D 

In [ ]:
#understanding the logs would make how the game progressed -> getting the game data.
#HS replay has shit load of data for sure.
#

In [47]:
import os
import re

# Function to parse the Power.log
def parse_power_log(power_log_path):
    if not os.path.exists(power_log_path):
        print("Power log file does not exist.")
        return []

    events = []

    with open(power_log_path, 'r', encoding='utf-8') as log_file:
        for line in log_file:
            event = parse_power_line(line)
            if event:
                events.append(event)

    return events

def parse_power_line(line):
    # Parse attacks
    if "ACTION_START" in line and "SubType=ATTACK" in line:
        match = re.search(r'Entity=\[entityName=(.+?) id=\d+ zone=.+? SubType=ATTACK .+? Target=\[entityName=(.+?) id=\d+ zone=', line)
        if match:
            attacker = match.group(1)
            target = match.group(2)
            return f"Attack - Attacker: {attacker}, Target: {target}"

    # Parse damage
    if "TAG_CHANGE" in line and "tag=DAMAGE" in line:
        match = re.search(r'TAG_CHANGE Entity=\[entityName=(.+?) id=\d+ .+?\] tag=DAMAGE value=(\d+)', line)
        if match:
            entity = match.group(1)
            damage = match.group(2)
            return f"Damage - Entity: {entity}, Damage: {damage}"

    # Parse card plays
    if "FULL_ENTITY" in line and "Creating ID" in line:
        match = re.search(r'Creating ID=\d+ CardID=(.+)', line)
        if match:
            card_id = match.group(1)
            return f"Card Play - Card ID: {card_id}"

    return None

# Function to parse the Zone.log
def parse_zone_log(zone_log_path):
    if not os.path.exists(zone_log_path):
        print("Zone log file does not exist.")
        return []

    events = []

    with open(zone_log_path, 'r', encoding='utf-8') as log_file:
        for line in log_file:
            event = parse_zone_line(line)
            if event:
                events.append(event)

    return events

def parse_zone_line(line):
    # Parse card draws
    if "zone=DECK" in line and "zone=HAND" in line:
        match = re.search(r'\[name=(.+?) id=\d+ zone=DECK .+?\] -> \[name=(.+?) id=\d+ zone=HAND .+? cardId=(.+?) entityType', line)
        if match:
            player_from = match.group(1)
            player_to = match.group(2)
            card_id = match.group(3)
            return f"Card Draw - Player: {player_to}, Card ID: {card_id}"

    # Parse card plays
    if "zone=HAND" in line and "zone=PLAY" in line:
        match = re.search(r'\[name=(.+?) id=\d+ zone=HAND .+?\] -> \[name=(.+?) id=\d+ zone=PLAY .+? cardId=(.+?) entityType', line)
        if match:
            player_from = match.group(1)
            player_to = match.group(2)
            card_id = match.group(3)
            return f"Card Play - Player: {player_to}, Card ID: {card_id}"

    # Parse card moves to graveyard
    if "zone=PLAY" in line and "zone=GRAVEYARD" in line:
        match = re.search(r'\[name=(.+?) id=\d+ zone=PLAY .+?\] -> \[name=(.+?) id=\d+ zone=GRAVEYARD .+? cardId=(.+?) entityType', line)
        if match:
            player_from = match.group(1)
            player_to = match.group(2)
            card_id = match.group(3)
            return f"Card Move to Graveyard - Player: {player_to}, Card ID: {card_id}"

    return None

In [52]:
print(power_events[10])

Damage - Entity: 돌가죽 방어구 제작자, Damage: 3


In [48]:
power_log_path=os.path.join(log_path, "Power.log")
zone_log_path=os.path.join(log_path, "Zone.log")

power_events = parse_power_log(power_log_path)
zone_events = parse_zone_log(zone_log_path)

all_events = power_events + zone_events
all_events.sort()  # Sort events if needed based on time (this is basic, further refinement may be required)

for event in all_events:
    print(event)

Card Play - Card ID: CORE_BT_117
Card Play - Card ID: CORE_CFM_790
Card Play - Card ID: CORE_CFM_790
Card Play - Card ID: CORE_EX1_383
Card Play - Card ID: CORE_RLK_741
Card Play - Card ID: CS2_108
Card Play - Card ID: ETC_113
Card Play - Card ID: ETC_113
Card Play - Card ID: ETC_526
Card Play - Card ID: EX1_407
Card Play - Card ID: EX1_606
Card Play - Card ID: HERO_01
Card Play - Card ID: HERO_01ao_Annhylde_hls
Card Play - Card ID: HERO_01bp
Card Play - Card ID: HERO_01dbp
Card Play - Card ID: REV_COIN2
Card Play - Card ID: REV_COIN2
Card Play - Card ID: REV_COIN2
Card Play - Card ID: REV_COIN2
Card Play - Card ID: TOY_330t5
Card Play - Card ID: TOY_330t5
Card Play - Card ID: TOY_607
Card Play - Card ID: TOY_907
Card Play - Card ID: TOY_907
Card Play - Card ID: TTN_060t
Card Play - Card ID: TTN_060t1
Card Play - Card ID: TTN_060t10t1
Card Play - Card ID: TTN_060t2
Card Play - Card ID: TTN_060t2
Card Play - Card ID: TTN_060t2
Card Play - Card ID: TTN_060t3
Card Play - Card ID: TTN_060t